# Linear Regression

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/* 

## Learning the TensorFlow way of linear regression

In [ ]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

housing_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
path = tf.keras.utils.get_file(housing_url.split("/")[-1], housing_url)

columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = pd.read_table(path, delim_whitespace=True, header=None, names=columns)

np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = train['MEDV']
train.drop('MEDV', axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = test['MEDV']
test.drop('MEDV', axis=1, inplace=True)

In [ ]:
def define_feature_columns(data_df, categorical_cols, numeric_cols):
    feature_columns = []
    
    for feature_name in numeric_cols:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
    
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

    return feature_columns

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=256):
    
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    
    return input_function

In [ ]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns = define_feature_columns(data, categorical_cols, numeric_cols)

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

In [ ]:
output_dir = './logs/LinearRegressor_1'
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns,
                                          model_dir = output_dir,
                                          config=tf.estimator.RunConfig().replace(save_summary_steps=100))
linear_est.train(train_input_fn)
result = linear_est.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir ./logs/LinearRegressor_1
# http://localhost:6006/

### Interactions

In [ ]:
def create_interactions(interactions_list, buckets=5):
    interactions = list()
    for (a, b) in interactions_list:
        interactions.append(tf.feature_column.crossed_column([a, b], hash_bucket_size=buckets))
    return interactions

derived_feature_columns = create_interactions([['RM', 'LSTAT']])

output_dir = './logs/LinearRegressor_2'
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns+derived_feature_columns,
                                          model_dir = output_dir,
                                          config=tf.estimator.RunConfig().replace(save_summary_steps=100))

linear_est.train(train_input_fn)

result = linear_est.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir ./logs/LinearRegressor_2

In [ ]:
def dicts_to_preds(pred_dicts):
    return np.array([pred['predictions'] for pred in pred_dicts])

preds = dicts_to_preds(linear_est.predict(test_input_fn))
print(preds)

### Keras to estimator version

In [ ]:
import tensorflow.keras as keras

In [ ]:
def define_feature_columns_layers(data_df, categorical_cols, numeric_cols):
    feature_columns = []
    feature_layer_inputs = {}
    
    for feature_name in numeric_cols:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name)
        
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        cat = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
        cat_one_hot = tf.feature_column.indicator_column(cat)
        feature_columns.append(cat_one_hot)
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name, dtype=tf.int32)
    
    return feature_columns, feature_layer_inputs

In [ ]:
def create_interactions(interactions_list, buckets=5):
    feature_columns = []
    
    for (a, b) in interactions_list:
        crossed_feature = tf.feature_column.crossed_column([a, b], hash_bucket_size=buckets)
        crossed_feature_one_hot = tf.feature_column.indicator_column(crossed_feature)
        feature_columns.append(crossed_feature_one_hot)
        
    return feature_columns

In [ ]:
def create_linreg(feature_columns, feature_layer_inputs, optimizer):

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, kernel_initializer='normal', activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [ ]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_linreg(feature_columns, feature_layer_inputs, optimizer)

In [ ]:
import tempfile

def canned_keras(model, output_dir = None):
    if output_dir is None:
        model_dir = tempfile.mkdtemp()
    else:
        model_dir = output_dir
    keras_estimator = tf.keras.estimator.model_to_estimator(
        keras_model=model, 
        model_dir=model_dir,
        config=tf.estimator.RunConfig().replace(save_summary_steps=100))
    return keras_estimator

estimator = canned_keras(model, output_dir='./logs/canned_1')

In [ ]:
train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir ./logs/canned_1

## Understanding coefficients in linear regression

In [ ]:
weights = estimator.get_variable_value('layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE')

In [ ]:
print(weights)

In [ ]:
def extract_labels(feature_columns):
    labels = list()
    for col in feature_columns:
        col_config = col.get_config()
        if 'key' in col_config:
            labels.append(col_config['key'])
        elif 'categorical_column' in col_config:
            if col_config['categorical_column']['class_name']=='VocabularyListCategoricalColumn':
                key = col_config['categorical_column']['config']['key']
                for item in col_config['categorical_column']['config']['vocabulary_list']:
                     labels.append(key+'_val='+str(item))
            elif col_config['categorical_column']['class_name']=='CrossedColumn':
                keys = col_config['categorical_column']['config']['keys']
                for bucket in range(col_config['categorical_column']['config']['hash_bucket_size']):
                    labels.append('x'.join(keys)+'_bkt_'+str(bucket))
    return labels

In [ ]:
labels = extract_labels(feature_columns)

for label, weight in zip(labels, weights):
    print(f"{label:15s} : {weight[0]:+.2f}")

## Understanding loss functions in linear regression

In [ ]:
def create_linreg(feature_columns, feature_layer_inputs, optimizer, 
                  loss='mean_squared_error', metrics=['mean_absolute_error']):

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, kernel_initializer='normal', activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [ ]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_linreg(feature_columns, feature_layer_inputs, optimizer,
                      loss='mean_absolute_error', 
                      metrics=['mean_absolute_error', 'mean_squared_error'])

estimator = canned_keras(model, output_dir='./logs/canned_2')

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir ./logs/canned_2

## Implementing lasso and ridge regression

In [ ]:
def create_ridge_linreg(feature_columns, feature_layer_inputs, optimizer, 
                        loss='mean_squared_error', metrics=['mean_absolute_error'],
                        l2=0.01):
    
    regularizer = keras.regularizers.l2(l2)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [ ]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_ridge_linreg(feature_columns, feature_layer_inputs, optimizer,
                      loss='mean_squared_error', 
                      metrics=['mean_absolute_error', 'mean_squared_error'],
                           l2=0.01)

estimator = canned_keras(model, output_dir='./logs/ridge')

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir './logs/ridge'

In [ ]:
def create_lasso_linreg(feature_columns, feature_layer_inputs, optimizer, 
                        loss='mean_squared_error', metrics=['mean_absolute_error'],
                        l1=0.001):
    
    regularizer = keras.regularizers.l1(l1)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [ ]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_lasso_linreg(feature_columns, feature_layer_inputs, optimizer,
                      loss='mean_squared_error', 
                      metrics=['mean_absolute_error', 'mean_squared_error'],
                           l1=0.001)

estimator = canned_keras(model, output_dir='./logs/lasso')

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir './logs/lasso'

## Implementing elastic net regression

In [ ]:
def create_elasticnet_linreg(feature_columns, feature_layer_inputs, optimizer, 
                        loss='mean_squared_error', metrics=['mean_absolute_error'],
                        l1=0.001, l2=0.01):
    
    regularizer = keras.regularizers.l1_l2(l1=l1, l2=l2)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='linear')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [ ]:
categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)
interactions_columns = create_interactions([['RM', 'LSTAT']])

feature_columns += interactions_columns

optimizer = keras.optimizers.Ftrl(learning_rate=0.02)
model = create_elasticnet_linreg(feature_columns, feature_layer_inputs, optimizer,
                      loss='mean_squared_error', 
                      metrics=['mean_absolute_error', 'mean_squared_error'],
                           l1=0.001, l2=0.01)

estimator = canned_keras(model, output_dir='./logs/elasticnet')

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir './logs/elasticnet'

## Implementing logistic regression

In [ ]:
breast_cancer = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'
path = tf.keras.utils.get_file(breast_cancer.split("/")[-1], breast_cancer)

columns = ['sample_code', 'clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
           'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
           'normal_nucleoli', 'mitoses', 'class']

data = pd.read_csv(path, header=None, names=columns, na_values=[np.nan, '?'])
data = data.fillna(data.median())

np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = (train['class']==4).astype(int)
train.drop(['sample_code', 'class'], axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = (test['class']==4).astype(int)
test.drop(['sample_code', 'class'], axis=1, inplace=True)

In [ ]:
def create_logreg(feature_columns, feature_layer_inputs, optimizer, 
                  loss='binary_crossentropy', metrics=['accuracy'],
                  l2=0.01):
    
    regularizer = keras.regularizers.l2(l2)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='sigmoid')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [ ]:
categorical_cols = []
numeric_cols = ['clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
                'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
                'normal_nucleoli', 'mitoses']

feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)

optimizer = keras.optimizers.Ftrl(learning_rate=0.007)
model = create_logreg(feature_columns, feature_layer_inputs, optimizer, l2=0.01)

estimator = canned_keras(model, output_dir='./logs/logreg')

train_input_fn = make_input_fn(train, y_train, num_epochs=300, batch_size=32)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir './logs/logreg'

## Resorting to non-linear regression

In [ ]:
try:
    from tensorflow.python.keras.layers.kernelized import RandomFourierFeatures
except:
    # from TF 2.2
    from tensorflow.keras.layers.experimental import RandomFourierFeatures 

In [ ]:
def create_svc(feature_columns, feature_layer_inputs, optimizer, 
               loss='hinge', metrics=['accuracy'],
               l2=0.01, output_dim=64, scale=None):
    
    regularizer = keras.regularizers.l2(l2)

    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    rff = RandomFourierFeatures(output_dim=output_dim, scale=scale, kernel_initializer='gaussian')(norm)
    outputs = keras.layers.Dense(1, 
                                 kernel_initializer='normal', 
                                 kernel_regularizer = regularizer, 
                                 activation='sigmoid')(rff)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [ ]:
!rm -rf ./log/rff

In [ ]:
categorical_cols = []
numeric_cols = ['clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
                'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
                'normal_nucleoli', 'mitoses']

feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)

optimizer = keras.optimizers.Adam(learning_rate=0.00005)
model = create_svc(feature_columns, feature_layer_inputs, optimizer, 
                   loss='hinge', l2=0.001, output_dim=512)

estimator = canned_keras(model, output_dir='./log/rff')

train_input_fn = make_input_fn(train, y_train, num_epochs=500, batch_size=512)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

In [ ]:
%tensorboard --logdir './log/rff'

## Preparing Wide & Deep Learning for Recommender Systems 

In [ ]:
census_dir = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/'
train_path = tf.keras.utils.get_file('adult.data', census_dir + 'adult.data')
test_path = tf.keras.utils.get_file('adult.test', census_dir + 'adult.test')

columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
           'marital_status', 'occupation', 'relationship', 'race', 'gender',
           'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
           'income_bracket']

train_data = pd.read_csv(train_path, header=None, names=columns)
test_data = pd.read_csv(test_path, header=None, names=columns, skiprows=1)

In [ ]:
predictors = ['age', 'workclass', 'education', 'education_num',
              'marital_status', 'occupation', 'relationship', 'gender']

y_train = (train_data.income_bracket==' >50K').astype(int)
y_test = (test_data.income_bracket==' >50K.').astype(int)

train_data = train_data[predictors]
test_data = test_data[predictors]

In [ ]:
train_data[['age', 'education_num']] = train_data[['age', 'education_num']].fillna(train_data[['age', 'education_num']].mean())
test_data[['age', 'education_num']] = test_data[['age', 'education_num']].fillna(train_data[['age', 'education_num']].mean())

In [ ]:
def define_feature_columns(data_df, numeric_cols, categorical_cols, categorical_embeds, dimension=30):
    numeric_columns = []
    categorical_columns = []
    embeddings = []
    
    for feature_name in numeric_cols:
        numeric_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
    
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        categorical_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
        
    for feature_name in categorical_embeds:
        vocabulary = data_df[feature_name].unique()
        to_categorical = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
        embeddings.append(tf.feature_column.embedding_column(to_categorical, dimension=dimension))
        
    return numeric_columns, categorical_columns, embeddings

def create_interactions(interactions_list, buckets=10):
    feature_columns = []
    
    for (a, b) in interactions_list:
        crossed_feature = tf.feature_column.crossed_column([a, b], hash_bucket_size=buckets)
        crossed_feature_one_hot = tf.feature_column.indicator_column(crossed_feature)
        feature_columns.append(crossed_feature_one_hot)
        
    return feature_columns

In [ ]:
numeric_columns, categorical_columns, embeddings = define_feature_columns(train_data, 
                                                                          numeric_cols=['age', 'education_num'], 
                                                                          categorical_cols=['gender'], 
                                                                          categorical_embeds=['workclass', 'education',
                                                                                              'marital_status', 'occupation', 
                                                                                              'relationship'], 
                                                                          dimension=32)

interactions = create_interactions([['education', 'occupation']], buckets=10)

In [ ]:
! rm -rf './logs/wd'

In [ ]:
output_dir = './logs/wd'

estimator = tf.estimator.DNNLinearCombinedClassifier(
    # wide settings
    linear_feature_columns=numeric_columns+categorical_columns+interactions,
    linear_optimizer=keras.optimizers.Ftrl(learning_rate=0.0002),
    # deep settings
    dnn_feature_columns=embeddings,
    dnn_hidden_units=[1024, 256, 128, 64],
    dnn_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    model_dir = output_dir,
    config=tf.estimator.RunConfig().replace(save_summary_steps=100))

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=256):
    
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    
    return input_function

In [ ]:
train_input_fn = make_input_fn(train_data, y_train, num_epochs=100, batch_size=256)
test_input_fn = make_input_fn(test_data, y_test, num_epochs=1, shuffle=False)

In [ ]:
estimator.train(input_fn=train_input_fn, steps=1500)

In [ ]:
results = estimator.evaluate(input_fn=test_input_fn)
print(results)

In [ ]:
def predict_proba(predictor):
    preds = list()
    for pred in predictor:
        preds.append(pred['probabilities'])
    return np.array(preds)
http://localhost:8888/notebooks/4.%20Linear%20Regression%20with%20TensorBoard.ipynb#
predictions = predict_proba(estimator.predict(input_fn=test_input_fn))

In [ ]:
predictions

In [ ]:
%tensorboard --logdir ./logs/wd